In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import xarray as xr
import numpy as np

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..'))

# teslakit
from teslakit.database import Database
from teslakit.storms import Extract_Circle
from teslakit.plotting.storms import WorldMap_Storms
from teslakit.statistical import CopulaSimulation



## Database and Site parameters

In [2]:
# --------------------------------------
# Teslakit database

p_data = r'/Users/nico/Projects/TESLA-kit/TeslaKit/data'
db = Database(p_data)

# set site
db.SetSite('KWAJALEIN')


In [3]:
# --------------------------------------
# load data and set parameters

xds_wmo = db.Load_TCs_noaa()  # noaa Allstorms.ibtracs_wmo

# wave point longitude and latitude
pnt_lon = 167.5
pnt_lat = 9.75

# radius for TCs selection (º)
r1 = 14
r2 = 4



## Historical TCs - extraction

In [4]:
# --------------------------------------
# Select Historical TCs inside circle

# dictionary with needed variable names 
d_vns = {
    'longitude': 'lon_wmo',
    'latitude': 'lat_wmo',
    'time': 'time_wmo',
    'pressure': 'pres_wmo',
}

# Select TCs that crosses a circular area R1
xds_TCs_r1, xds_TCs_r1_params = Extract_Circle(xds_wmo, pnt_lon, pnt_lat, r1, d_vns)

# Select TCs that crosses a circular area R2
xds_TCs_r2, xds_TCs_r2_params = Extract_Circle(xds_wmo, pnt_lon, pnt_lat, r2, d_vns)

print(xds_TCs_r1)
print()
print(xds_TCs_r2)

# store data
db.Save_TCs_r1(xds_TCs_r1, xds_TCs_r1_params)
db.Save_TCs_r2(xds_TCs_r2, xds_TCs_r2_params)


<xarray.Dataset>
Dimensions:          (center: 26, storm: 309, time: 137)
Dimensions without coordinates: center, storm, time
Data variables:
    storm_sn         (storm) |S13 ...
    name             (storm) |S57 ...
    numObs           (storm) int16 ...
    season           (storm) int16 ...
    track_type       (storm) int8 ...
    genesis_basin    (storm) int8 ...
    num_basins       (storm) int8 ...
    basin            (storm, time) float32 ...
    wind_avg_period  (center) float32 ...
    source           (center) |S10 ...
    time_wmo         (storm, time) datetime64[ns] 1951-03-18T06:00:00.000043520 ... NaT
    lat_wmo          (storm, time) float32 5.7 6.0 6.3999996 6.7 ... nan nan nan
    lon_wmo          (storm, time) float32 158.3 159.4 160.4 ... nan nan nan
    alt              (storm, time) float32 ...
    wind_wmo         (storm, time) float32 ...
    pres_wmo         (storm, time) float32 1002.0 1002.0 1000.0 ... nan nan nan
    sub_basin        (storm, time) float32

In [ ]:
# TODO: plot historical TCs inside both circles